# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [3]:
import os
import pandas as pd
import numpy as np
import re
import json 
PUIdata = os.getenv('PUIDATA')

In [30]:
# Load dataset
#!curl -O https://nycopendata.socrata.com/api/views/erm2-nwe9/rows.csv?accessType=DOWNLOAD
#jfile = open('https://data.cityofnewyork.us/resource/fhrw-4uyv.json')

#data = json.loads('https://data.cityofnewyork.us/resource/fhrw-4uyv.json')

#query = ("https://data.cityofnewyork.us/resource/fhrw-4uyv.json")
#raw_data = pd.read_json(query)

#data = pd.read_csv('https://nycopendata.socrata.com/api/views/erm2-nwe9/rows.csv?accessType=DOWNLOAD', usecols=[0,1,2,3])

data = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/311_Service_Requests_from_2010_to_Present.csv'\
                  , usecols=['Created Date','Complaint Type','Community Board', 'Borough', 'Resolution Description'])

In [31]:
#data = DataFrame(rsw_data)
data.head()

,Created Date,Complaint Type,Resolution Description,Community Board,Borough
0,09/01/2016 12:00:06 AM,Noise - Residential,The Police Department responded to the complai...,13 QUEENS,QUEENS
1,09/01/2016 12:00:35 AM,Blocked Driveway,The Police Department reviewed your complaint ...,12 BRONX,BRONX
2,09/01/2016 12:00:37 AM,Blocked Driveway,The Police Department responded and upon arriv...,03 QUEENS,QUEENS
3,09/01/2016 12:00:54 AM,Blocked Driveway,The Police Department issued a summons in resp...,12 BRONX,BRONX
4,09/01/2016 12:01:15 AM,Street Condition,The Department of Transportation inspected thi...,12 MANHATTAN,MANHATTAN


In [32]:
data.shape


(34897, 5)

# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [33]:
# Check if all Boroughs and Community Districts are represented in the Data 

print 'All boroughs are represented in the data: ', data.Borough.unique()
print 'Community Boards represented in the data: ', data['Community Board'].unique()
print 'No not all community boards are represented in the data. Infact, \
community boards represented seem to be mislabeled.'


All boroughs are represented in the data:  ['QUEENS' 'BRONX' 'MANHATTAN' 'BROOKLYN' 'Unspecified' 'STATEN ISLAND']
Community Boards represented in the data:  ['13 QUEENS' '12 BRONX' '03 QUEENS' '12 MANHATTAN' '09 MANHATTAN'
 '08 QUEENS' '01 QUEENS' '07 BRONX' '01 BROOKLYN' '03 MANHATTAN'
 '09 QUEENS' '05 BRONX' '03 BROOKLYN' '07 MANHATTAN' '12 BROOKLYN'
 '0 Unspecified' '09 BROOKLYN' '07 QUEENS' '15 BROOKLYN' '08 MANHATTAN'
 '10 BROOKLYN' '08 BROOKLYN' '04 MANHATTAN' '02 MANHATTAN' '12 QUEENS'
 '04 BRONX' '04 BROOKLYN' '01 BRONX' 'Unspecified MANHATTAN' '10 QUEENS'
 '18 BROOKLYN' '11 BROOKLYN' '10 MANHATTAN' '11 QUEENS' '16 BROOKLYN'
 '05 BROOKLYN' '05 MANHATTAN' '55 BROOKLYN' '08 BRONX' '09 BRONX'
 '17 BROOKLYN' '01 MANHATTAN' '05 QUEENS' '06 QUEENS' '02 BRONX' '11 BRONX'
 '04 QUEENS' 'Unspecified BROOKLYN' '02 QUEENS' '01 STATEN ISLAND'
 '03 STATEN ISLAND' '14 BROOKLYN' '07 BROOKLYN' '02 STATEN ISLAND'
 '10 BRONX' '11 MANHATTAN' '06 BROOKLYN' '13 BROOKLYN' '06 MANHATTAN'
 'Unspecifie

In [34]:
# How many unique values do we have? 
print 'Unique NYC boroughs represented in the data: ', len(data.Borough.unique())
print 'Unique NYC Community Boards represented in the data: ', len(data['Community Board'].unique())


Unique NYC boroughs represented in the data:  6
Unique NYC Community Boards represented in the data:  74


In [35]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.

print 'Survey Error'


Survey Error


In [36]:
# Check for duplicates? Are these plausible?

#df.groupby(['Letters', 'Numbers']).size()

data.groupby(['Created Date', 'Complaint Type', 'Resolution Description', 'Community Board', 'Borough']).size()


Created Date            Complaint Type                     Resolution Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              Community Board    Borough      
09/01/2016 01:00:00 PM  Sweeping/Inadequate                The Department of Sanitation has investigated the complaint and addressed the issue. If the problem persists, call 311 to enter a new complaint. If you are outside of New York City, please call (212) NEW-YORK (212-639-9675).                                                                                                                             

In [37]:
print("Total number of duplicated values %s"%data.duplicated().sum())

Total number of duplicated values 1592


In [21]:
print 'As per the output above, we can see many duplicate rows and many rows with missing fields.'

As per the output above, we can see many duplicate rows and many rows with missing fields.


In [38]:
# What about missing values? Can you detect any patterns? 

print 'Yes, many rows have missing Complaint Type and Resolution Descriptions. No specific patterns.'

Yes, many rows have missing Complaint Type and Resolution Descriptions. No specific patterns.


In [43]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?

duplicates = pd.Series(data['Created Date'].duplicated())
duplicates.head()

0    False
1    False
2    False
3    False
4    False
Name: Created Date, dtype: bool

In [84]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
c_boards = pd.Series(data['Community Board'])
data['c_num'] = c_boards.str.split(' ', expand=True)
#data['c_num2'] = data.c_num.astype(float)
#data['c_bour'] = c_boards.str.split(' ', expand=True)
data = data.drop(data[data.c_num  == 'Unspecified'])
pd.to_numeric(data['c_num'])

data.head(50)



ValueError: labels ['Created Date' 'Complaint Type' 'Resolution Description' 'Community Board'
 'Borough' 'c_num'] not contained in axis

In [74]:
# Drop the marked districts

data = data.drop(data[data.c_num > 12 & data.Borough == 'BRONX'])

TypeError: cannot compare a dtyped [object] array with a scalar of type [bool]

In [ ]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level



In [ ]:
# Safe reduced data frame (Community District level)



# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [86]:
# Population by Community District
#df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")

df_pop = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')

In [94]:
# Check variables in file

df_pop.head()


,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [91]:
# How many community districts are in file? 
cd = len(df_pop.cd_id.unique())
print 'There are', cd, 'community districts represented in the file.'

There are 59 community districts represented in the file.


In [95]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311

df_pop.groupby(['cd_id'])


In [ ]:
# Save data frame

In [ ]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [ ]:
# Check variables in file

In [ ]:
# How many community districts are in file? 

In [ ]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data

In [ ]:
# Aggregate internet type by high and low connections

In [ ]:
# Save data frame 

# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [ ]:
# Harmonize identifier of dataframe 1

In [ ]:
# Harmonize identifier of dataframe 2

In [ ]:
# Harmonize identifier of dataframe 3

In [ ]:
# Link the 3 dataframes

In [ ]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?